in this notebook, for a given runlist, the error estimation is done

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import yaml

from gammapy.data import DataStore

import warnings
warnings.filterwarnings('ignore')

# ADD HERE YOUR INTEREST

If you are following this repository and you already have a source folder, check that source_flag is True and add the source name.

Else, check that source_flag is False, and add the runslist for what you want to calculate the error and add the output path for it.

NOTE: the outputfile will give you the systematic error from 0.1-100TeV divided into 24 bins in units of statistical std.
if you want it in units of bkg percentage, multiply this result by 1/sqrt(bkg_i), 
where bkg_i is the number of bkg events in the entire off region (whole FoV excluded the gamma sources)

In [2]:
source_flag= True

if source_flag:
    source='Crab'
    obsid_list = np.loadtxt(f'{source}/runlist.txt').astype(int)
    output_path=source
else:
    obsid_list = np.loadtxt(f'{source}/runlist.txt').astype(int)
    output_path= None

In [3]:
with open("../general_config.yml", "r") as ymlfile:
    cfg = yaml.load(ymlfile, Loader=yaml.FullLoader)
conf=cfg['conf']
repo_path=cfg['repo_path']
N_ebins = cfg['N_ebins']
zen_bins = cfg['zen_bins']

hesseras = ['hess1', 'hess2']
model_str = ['B', 'D', 'C']
energy_bins = np.logspace(-1, 2, N_ebins+1)

In [4]:
# getting the zenith pointing of them

error_list = []
for hessera in hesseras:
    basedir = f'$FITS_PROD/{hessera}/std_{conf}_fullEnclosure'
    ds = DataStore.from_dir(basedir, f'hdu-index-bg-latest-fov-radec.fits.gz', f'obs-index-bg-latest-fov-radec.fits.gz')
    table = ds.obs_table
    mask = [True if obsid in obsid_list else False for obsid in table['OBS_ID']]
    
    if np.sum(mask) == 0:
        print(f'no runs for {hessera}')
    else:
        table=table[mask]

        ### opening the error estimation
        er_est = np.ndarray((7, 3, 24))    
        for model in range(3):
            if hessera == 'hess1':
                er_est[:, model] = np.loadtxt(f'../fixed_material/{hessera}_error_estimation_{model_str[model]}.txt')
            else:
                er_est[:, model] = np.loadtxt(f'../fixed_material/{hessera}_error_estimation_C.txt')
                
        
        setting=np.ndarray((len(table), 2)) # the first parameter is for zen bin and second for muoneff

        for zen_idx in range(7):
            mask_zen1 = table['ZEN_PNT'] > zen_bins[zen_idx]
            mask_zen2 = table['ZEN_PNT'] < zen_bins[zen_idx+1]
            mask_zen = mask_zen1 & mask_zen2
            setting[mask_zen, 0] = zen_idx

        mask_muoneff1 = table['MUONEFF'] > 0.085
        setting[mask_muoneff1, 1] = 0
        mask_muoneff2 = table['MUONEFF'] < 0.065
        setting[mask_muoneff2, 1] = 2
        mask_muoneff3 = mask_muoneff1 + mask_muoneff2
        setting[~mask_muoneff3, 1] = 1
        
        error_hessera = np.zeros((N_ebins))
        for e_idx in range(N_ebins):
            error2_aux = 0

            for az in [0, 1]:
                for zen in range(7):
                    for model in range(3):
                        m1= setting[:, 0] == zen
                        m2 = setting[:, 1] == model
                        m = m1 & m2
                        N_runs = np.sum(m)
                        if ~np.isnan(er_est[zen][model][e_idx]) and N_runs != 0:
                            error2_aux += N_runs * (er_est[zen][model][e_idx] ** 2)

            error_hessera[e_idx] = np.sqrt(error2_aux)
        error_list.append(error_hessera)

In [5]:
for i, error in enumerate(error_list):
    if i==0:
        error_final = error**2
    else:
        error_final += error**2  
        
for e_idx in range(N_ebins):
    print(f'energy range:{energy_bins[e_idx]:.3f}-{energy_bins[e_idx + 1]:.3f}, amplitude={np.sqrt(error_final[e_idx])} std')
    error_final[e_idx] = np.sqrt(error_final[e_idx])
np.savetxt(f'{output_path}/sysamplitude.txt', error_final)

energy range:0.100-0.133, amplitude=0.0 std
energy range:0.133-0.178, amplitude=0.0 std
energy range:0.178-0.237, amplitude=0.0 std
energy range:0.237-0.316, amplitude=0.0 std
energy range:0.316-0.422, amplitude=0.0 std
energy range:0.422-0.562, amplitude=0.0 std
energy range:0.562-0.750, amplitude=8.477074693454096 std
energy range:0.750-1.000, amplitude=20.171913603797993 std
energy range:1.000-1.334, amplitude=22.4797265056122 std
energy range:1.334-1.778, amplitude=7.022186458478061 std
energy range:1.778-2.371, amplitude=3.7532245357582643 std
energy range:2.371-3.162, amplitude=3.58199220458183 std
energy range:3.162-4.217, amplitude=3.1930699442026307 std
energy range:4.217-5.623, amplitude=11.00761341038469 std
energy range:5.623-7.499, amplitude=6.1914840787481475 std
energy range:7.499-10.000, amplitude=6.734989197880174 std
energy range:10.000-13.335, amplitude=6.655880600312998 std
energy range:13.335-17.783, amplitude=7.805634724241566 std
energy range:17.783-23.714, ampli